In [1]:
# imports
from pathlib import Path
from time import time

import numpy as np
import pandas as pd

from utils import (
    read_single_problem_from_path_as_adjacency,
    read_single_problem_from_path_as_sparse,
    read_single_problem_from_path_as_sparse_from_adjacency,
    order_nodes_in_descending_order,
    check_validity_for_adjacency,
    transform_node_clique_to_zero_one,
    transform_zero_one_clique_to_nodes
)
from constants import BASE_INSTANCES_FILES, OTHER_INSTANCES_FILES, OTHER_INSTANCES_BEST_KNOWN
from heuristics import (
    HeuristicRunner,
    descending_degree_glutonous_heuristic,
    dynamic_descending_degree_glutonous_heuristic,
    descending_degree_random_heuristic,
    multiple_descending_degree_random_heuristic
)

# Constants
ROOT_DIR = Path.cwd().parent
# Instances pathes
INSTANCES_DIR = ROOT_DIR / "instances"
BASE_INSTANCES_DIR = INSTANCES_DIR / "project_instances"
OTHER_INSTANCES_DIR = INSTANCES_DIR / "other_instances"


In [2]:
_, _, graph, degrees = read_single_problem_from_path_as_adjacency(
    instance_path=BASE_INSTANCES_DIR / "random-10.col"
)
print(graph)
clique = descending_degree_glutonous_heuristic(graph=graph, degrees=degrees)
print(clique)
print("Is the clique valid ? ", check_validity_for_adjacency(graph, clique))


[[0 1 0 1 1 1 0 0 1 0]
 [1 0 1 0 1 0 1 1 1 1]
 [0 1 0 1 0 1 0 0 0 1]
 [1 0 1 0 0 0 0 0 1 0]
 [1 1 0 0 0 1 1 1 0 1]
 [1 0 1 0 1 0 1 0 1 0]
 [0 1 0 0 1 1 0 1 1 1]
 [0 1 0 0 1 0 1 0 0 1]
 [1 1 0 1 0 1 1 0 0 0]
 [0 1 1 0 1 0 1 1 0 0]]
[0 1 0 0 1 0 1 1 0 1]
Is the clique valid ?  True


In [3]:
# Check that clique representation utils work
number_of_nodes, _, graph, degrees = read_single_problem_from_path_as_adjacency(
    instance_path=BASE_INSTANCES_DIR / "random-10.col"
)
clique = descending_degree_glutonous_heuristic(graph=graph, degrees=degrees)
print(clique)
print(transform_zero_one_clique_to_nodes(clique))
print(transform_node_clique_to_zero_one(number_of_nodes, transform_zero_one_clique_to_nodes(clique)))

[0 1 0 0 1 0 1 1 0 1]
[1, 4, 6, 7, 9]
[0 1 0 0 1 0 1 1 0 1]


In [4]:
# Test on project instances
runner = HeuristicRunner(
    instances_dir=BASE_INSTANCES_DIR,
    instances=BASE_INSTANCES_FILES,
    methods=[
        (descending_degree_glutonous_heuristic, "desc_base"),
        (dynamic_descending_degree_glutonous_heuristic, "desc_dyn"),
        (descending_degree_random_heuristic, "desc_ran"),
        (multiple_descending_degree_random_heuristic, "mult_ran")
    ],
    known_bests={},
    display_time_details=False
)
runner.run_all_methods_on_all_instances()
runner.display_results()

size_of_choice 5
size_of_choice 5
size_of_choice 5
size_of_choice 5
size_of_choice 5
size_of_choice 5
size_of_choice 5
size_of_choice 5
size_of_choice 5
size_of_choice 5
size_of_choice 5
size_of_choice 5
size_of_choice 5
size_of_choice 5
size_of_choice 5
size_of_choice 5
size_of_choice 5
size_of_choice 5
size_of_choice 5
size_of_choice 5
size_of_choice 5
size_of_choice 5
size_of_choice 5
size_of_choice 5
size_of_choice 5
size_of_choice 5
size_of_choice 5
size_of_choice 5
size_of_choice 5
size_of_choice 5
size_of_choice 5
size_of_choice 5
size_of_choice 5
size_of_choice 5
size_of_choice 5
size_of_choice 5
size_of_choice 5
size_of_choice 5
size_of_choice 5
size_of_choice 5
size_of_choice 5
size_of_choice 5
size_of_choice 5
size_of_choice 5
size_of_choice 5
size_of_choice 5
size_of_choice 5
size_of_choice 5
size_of_choice 5
size_of_choice 5
size_of_choice 5
size_of_choice 5
size_of_choice 5
size_of_choice 5
size_of_choice 5
size_of_choice 5
size_of_choice 5
size_of_choice 5
size_of_choice

In [5]:
# Test on other bigger instances.
runner = HeuristicRunner(
    instances_dir=OTHER_INSTANCES_DIR,
    instances=OTHER_INSTANCES_FILES,
    methods=[
        (descending_degree_glutonous_heuristic, "desc_base"),
        (dynamic_descending_degree_glutonous_heuristic, "desc_dyn"),
        (descending_degree_random_heuristic, "desc_ran"),
        (multiple_descending_degree_random_heuristic, "mult_ran")
    ],
    known_bests=OTHER_INSTANCES_BEST_KNOWN,
    display_time_details=False
)
runner.run_all_methods_on_all_instances()
runner.display_results()

size_of_choice 5
size_of_choice 5
size_of_choice 5
size_of_choice 5
size_of_choice 5
size_of_choice 5
size_of_choice 5
size_of_choice 5
size_of_choice 5
size_of_choice 5
size_of_choice 5
size_of_choice 5
size_of_choice 5
size_of_choice 5
size_of_choice 5
size_of_choice 5
size_of_choice 5
size_of_choice 5
size_of_choice 5
size_of_choice 5
size_of_choice 5
size_of_choice 5
size_of_choice 5
size_of_choice 5
size_of_choice 5
size_of_choice 5
size_of_choice 5
size_of_choice 5
size_of_choice 5
size_of_choice 5
size_of_choice 5
size_of_choice 5
size_of_choice 5
size_of_choice 5
size_of_choice 5
size_of_choice 5
size_of_choice 5
size_of_choice 5
size_of_choice 5
size_of_choice 5
size_of_choice 5
size_of_choice 5
size_of_choice 5
size_of_choice 5
size_of_choice 5
size_of_choice 5
size_of_choice 5
size_of_choice 5
size_of_choice 5
size_of_choice 5
size_of_choice 5
size_of_choice 5
size_of_choice 5
size_of_choice 5
size_of_choice 5
size_of_choice 5
size_of_choice 5
size_of_choice 5
size_of_choice

In [7]:
# Test different size of choices for the nodes for random heuristic
methods = []
for k in range(2, 20):
    methods.append((lambda *args, **kwargs: descending_degree_random_heuristic(*args, **kwargs, size_of_choice=k), f"ran_{k}"))
print(methods)
runner = HeuristicRunner(
    instances_dir=BASE_INSTANCES_DIR,
    instances=BASE_INSTANCES_FILES,
    methods=methods,
    known_bests={},
    display_time_details=False
)
runner.run_all_methods_on_all_instances()
runner.display_results()

[(<function <lambda> at 0x0000024BFF18BAC0>, 'ran_2'), (<function <lambda> at 0x0000024BFF18A4D0>, 'ran_3'), (<function <lambda> at 0x0000024BFF18B9A0>, 'ran_4'), (<function <lambda> at 0x0000024BFF18BBE0>, 'ran_5'), (<function <lambda> at 0x0000024BFF18BC70>, 'ran_6'), (<function <lambda> at 0x0000024BFF18BD00>, 'ran_7'), (<function <lambda> at 0x0000024BFF18BD90>, 'ran_8'), (<function <lambda> at 0x0000024BFF18BE20>, 'ran_9'), (<function <lambda> at 0x0000024BFF18BEB0>, 'ran_10'), (<function <lambda> at 0x0000024BFF18BF40>, 'ran_11'), (<function <lambda> at 0x0000024BFF1F4040>, 'ran_12'), (<function <lambda> at 0x0000024BFF1F40D0>, 'ran_13'), (<function <lambda> at 0x0000024BFF1F4160>, 'ran_14'), (<function <lambda> at 0x0000024BFF1F41F0>, 'ran_15'), (<function <lambda> at 0x0000024BFF1F4280>, 'ran_16'), (<function <lambda> at 0x0000024BFF1F4310>, 'ran_17'), (<function <lambda> at 0x0000024BFF1F43A0>, 'ran_18'), (<function <lambda> at 0x0000024BFF1F4430>, 'ran_19')]
size_of_choice 19